# Задание 1. Прямые методы решения линейных систем

In [1]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [21]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


In [22]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A = ', A.tex(), ',~~', 'B = ', B.tex())
print_tex(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ A =  \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} ,~~ B =  \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} $$

$$ \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}
\end{pmatrix} = \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [23]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] = A_[j][i] - d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


print_system_output(A, 'A', B, SingleGauss(A, B, n), n)


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.678300 \\ -0.593793 \\ 0.157882 \\ 0.389392 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \begin{pmatrix} 0.678300 \\ -0.593793 \\ 0.157882 \\ 0.389392 \end{pmatrix} = \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
4.065893270275645\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458878
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
4.065893270275646\\
5.296215905463411\\
0.5578220990686975\\
0.17931857867458945
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
-8.881784197001252e-16\\
0.0\\
-8.881784197001252e-16\\
-6.661338147750939e-16
\end{pmatrix} $$

### Решение методом Гаусса с выбором главного элемента

In [24]:
def ModifiedGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
        X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
        for k in range(n-1, -1, -1):
            s = 0
            for i in range(k+1, n):
                s += AB[k][i] * X[i][0]
            X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

print_system_output(A, 'A', B, ModifiedGauss(A, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.678300 \\ -0.593793 \\ 0.157882 \\ 0.389392 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \begin{pmatrix} 0.678300 \\ -0.593793 \\ 0.157882 \\ 0.389392 \end{pmatrix} = \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
4.065893270275645\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458878
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
4.065893270275646\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458856
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
-8.881784197001252e-16\\
0.0\\
0.0\\
2.220446049250313e-16
\end{pmatrix} $$

### Решение методом $LU$-разложения

In [25]:
def GetLU(A: Matrix, n: int) -> Tuple[Matrix, Matrix]:
    L = Matrix(np.zeros((n, n)))
    U = Matrix(np.identity(n))
    for i in range(n):
        for j in range(i, n):
            s = 0
            for k in range(i):
                s += L[j][k] * U[k][i]
            L[j][i] = A[j][i] - s
        for j in range(i, n):
            s = 0
            for k in range(i):
                s += L[i][k] * U[k][j]
            U[i][j] = (A[i][j] - s) / L[i][i]
    return L, U

L, U = GetLU(A, n)
print_tex('L = ', L.tex())
print_tex('U = ', U.tex())
print_tex(r'L \times U = ', (np.matmul(L, U)).tex(), r'\stackrel{?}{=}', A.tex(), '= A')

$$ L =  \begin{pmatrix} 5.988915 & 0 & 0 & 0 \\ 8.602662 & 0.913006 & 0 & 0 \\ 2.104577 & 2.167480 & 17.347444 & 0 \\ 7.363161 & 3.519556 & 20.965554 & -8.296277 \end{pmatrix} $$

$$ U =  \begin{pmatrix} 1 & 0.829613 & 1.182736 & 0.787096 \\ 0 & 1 & -7.430370 & 3.007015 \\ 0 & 0 & 1 & -0.343155 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

$$ L \times U =  \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} = A $$

In [26]:
Y_ = SingleGauss(L, B, n)
X_ = SingleGauss(U, Y_, n)
print_system_output(A, 'A', B, X_, n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.678300 \\ -0.593793 \\ 0.157882 \\ 0.389392 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \begin{pmatrix} 0.678300 \\ -0.593793 \\ 0.157882 \\ 0.389392 \end{pmatrix} = \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
4.065893270275645\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458878
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
4.065893270275645\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458767
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
0.0\\
0.0\\
1.1102230246251565e-15
\end{pmatrix} $$

### Нахождение обратной к матрице $A$

In [27]:
def inverse(A: Matrix, n: int) -> Matrix:
    R = Matrix(np.zeros(shape=(n, 1)))
    A_ = Matrix(np.zeros(shape=(n, n)))
    for i in range(n):
        R[i][0] = 1.
        S = SingleGauss(A, R, n)
        for j in range(n):
            A_[j][i] = S[j][0]
        R[i][0] = 0
    return A_

A_ = inverse(A, n)
print_tex(r'A^{-1} = ', A_.tex())
print_tex(r'A \times A^{-1} = ', (np.matmul(A, A_)).tex())

def norm(M: Matrix) -> float:
    return max([sum([abs(M[i][j]) for i in range(n)]) for j in range(n)])

print_tex(r'||A||_1 * ||A^{-1}||_1 = ', str(norm(A) * norm(A_)))


$$ A^{-1} =  \begin{pmatrix} 0.236721 & 0.000113 & -0.542049 & 0.098071 \\ -0.229497 & 0.024104 & 0.361717 & 0.055115 \\ 0.151037 & -0.096077 & 0.107635 & -0.041363 \\ -0.073675 & 0.118820 & 0.145676 & -0.120536 \end{pmatrix} $$

$$ A \times A^{-1} =  \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

$$ ||A||_1 * ||A^{-1}||_1 =  30.731924037945404 $$

### Решение системы в случае $Cx=b~:~c_{11}=10^{-8}a_{11}, c_{ij}=a_{ij}~\forall i,j~:~(i,j)\ne(1,1)$

In [28]:
C = A.copy()
C[0][0] *= 1.e-8
print_tex('A=', A.tex())
print_tex('C=', C.tex())

$$ A= \begin{pmatrix} 5.988915 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} $$

$$ C= \begin{pmatrix} 0.000000 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} $$

In [33]:
print_system_output(C, 'C', B, SingleGauss(C, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} -1.623886 \\ 1.638137 \\ -1.311000 \\ 1.105901 \end{pmatrix} $$

$$ C \times X^* =  \begin{pmatrix} 0.000000 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \begin{pmatrix} -1.623886 \\ 1.638137 \\ -1.311000 \\ 1.105901 \end{pmatrix} = \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
4.065893270275645\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458878
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
4.0658932702756445\\
5.296215674480376\\
0.5578220901160105\\
0.17931873088917882
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
8.881784197001252e-16\\
2.309830353652842e-07\\
8.952686147978284e-09\\
-1.522145900434424e-07
\end{pmatrix} $$

In [31]:
print_system_output(C, 'C', B, ModifiedGauss(C, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} -1.623886 \\ 1.638137 \\ -1.311000 \\ 1.105901 \end{pmatrix} $$

$$ C \times X^* =  \begin{pmatrix} 0.000000 & 4.968483 & 7.083306 & 4.713851 \\ 8.602662 & 8.049889 & 3.390708 & 9.516544 \\ 2.104577 & 3.913465 & 3.731426 & 2.221286 \\ 7.363161 & 9.628132 & 3.522629 & 0.888161 \end{pmatrix} \begin{pmatrix} -1.623886 \\ 1.638137 \\ -1.311000 \\ 1.105901 \end{pmatrix} = \begin{pmatrix} 4.065893 \\ 5.296216 \\ 0.557822 \\ 0.179319 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
4.065893270275645\\
5.296215905463411\\
0.5578220990686966\\
0.17931857867458878
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
4.0658932702756445\\
5.296215905463409\\
0.5578220990686962\\
0.17931857867458945
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
8.881784197001252e-16\\
1.7763568394002505e-15\\
4.440892098500626e-16\\
-6.661338147750939e-16
\end{pmatrix} $$